# TODO

 - dependency parsing, e.g. with spaCy
 - 

In [9]:
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from ngrams import Ngram

In [2]:
df = pd.read_csv("../../../data/v0_2.csv.gz", 
                 dtype=dict(Provenance="string", RelatedWorks="string"))

# TODO: save & load DF s.t. these lines are not necessary here                
df["ObjectID"] = df.ObjectID.astype("int")
df = df.set_index("ObjectID")

In [3]:
docs = list(df.Title.dropna())
ng = Ngram(ns=2, documents=docs, precompute_freqs=True)

(1, 2)-grams: Padding documents...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135361/135361 [00:00<00:00, 1631292.30it/s]


(1, 2)-grams: Term Document Matrix constructed...
(1, 2)-grams: Term frequencies precomputed...
(1, 2)-grams: Init done


In [4]:
toks = [w for s in docs for w in s.split()]
len(toks), len(set(toks)), ng.N

(859789, 82796, 2140449)

In [ ]:
82796/859789, 82796/2140449, 2140449/82796**2

In [ ]:
# def pmi(w1, w2):
#     tup = w1 + " " + w2
#     return ng.prob(tup, log=True) - (ng.prob(w1, log=True)+ng.prob(w2, log=True))

# pmis = [[(w1, w2, pmi(w1, w2)) for w1, w2 in list(ng.iter_ngrams(s, as_tuples=True))]
#              for s in tqdm(docs[:10000])]

def pmi(tup):
    w1, w2 = tup.split()
    return ng.prob(tup, log=True) - (ng.prob(w1, log=True)+ng.prob(w2, log=True))
    
pmis = {w: pmi(w) for w in tqdm(ng.vocab(2))}

In [ ]:
_=plt.hist(pmis.values(), bins=50)

In [ ]:
more_than_once = {w: (pmi, ng.freq(w)) for w, pmi in pmis.items() 
#                   if ng.freq(w) > 1000}
                  if sum(map(ng.freq, w.split())) > 1000}
# sorted(more_than_once.items(), key=lambda tup: (tup[1][1], -tup[1][1]), reverse=True)[:100]

In [ ]:
ps, freqs = tuple(zip(*more_than_once.values()))

# plt.plot(pmis[:150000], freqs[:150000], ".")
plt.plot(ps, freqs, ".")
# plt.ylim((-5, 2000))

In [ ]:
ps = np.asarray(ps)

plt.hist((ps-np.mean(ps))/np.var(ps), bins=100)

In [ ]:
ng.prob("een van"), ng.prob("een")*ng.prob("van"),\
ng.prob("een van", log=True) - (ng.prob("een", log=True)+ng.prob("van", log=True))

In [ ]:
w, u = "nieuw", "amsterdam"
ng.prob(f"{w} {u}"), ng.prob(w)*ng.prob(u),\
ng.prob(f"{w} {u}", log=True) - (ng.prob(w, log=True)+ng.prob(u, log=True))

In [ ]:
avgs = []
maxs = []
stds = []
for d in tqdm(docs):
    pairs = list(ng.iter_ngrams(d, padding=False))
    scores = list(map(pmi, pairs))
    avgs.append(np.mean(scores))
    maxs.append(np.max(scores) if len(scores) else 0)
    stds.append(np.var(scores)**.5 if len(scores) else 0)
    

In [ ]:
lens = [len(list(ng.iter_ngrams(d, padding=False))) for d in tqdm(docs)]
sns.jointplot(fs, lens)

In [ ]:
sns.jointplot(avgs, maxs)

In [ ]:
# class PMI:
#     def __init__(self, texts, **model_params):
#         default_params = dict(ns=2, documents=texts, precompute_freqs=True)
#         default_params.update(model_params)
#         self.model = Ngram(**default_params)
        
#     def pmi(self, pair_str):
#         w1, w2 = pair_str.split()
#         return (
#                 self.model.prob(pair_str, log=True) - 
#                 (self.model.prob(w1, log=True) + 
#                  self.model.prob(w2, log=True))
#                )

#     def process_object(self, row):
#         pairs, pmis = [], []
#         for text in row:
#             cur_pairs = list(self.model.iter_ngrams(text, padding=False))
#             pairs.extend(cur_pairs)
#             pmis.extend(map(self.pmi, cur_pairs))
            
#         return sorted(zip(pairs, pmis), key=lambda t:t[1]), np.mean(pmis)
    
    
#     # distribution at q=99.5 is very narrow
#     # -> q=95 is broader and retains 90% of objects
#     @staticmethod
#     def percentile_norm(v, q=95):
#         return (v - np.percentile(v, 100-q))/(np.percentile(v, q) - np.percentile(v, 100-q))

#     # taking sqrt implies scores centered & symmetric around 0.5
#     # (otherwise high skew towards 0)
#     def norm(self, v, q=95, do_sqrt=False):
#         normed = self.percentile_norm(v, q)
#         return normed**0.5 if do_sqrt else normed
        
        
    
#     def process_objects(self, rows):
#         tuples = rows.progress_apply(
#                     axis='columns', 
#                     func=self.process_object
#                 )
        
#         scores = tuples.apply(lambda t: t[1])
#         scores = self.norm(scores, do_sqrt=True)
#         scores.name = "scores"
        
        
#         details = tuples.apply(lambda t: t[0])
#         all_vals = np.asarray([v for obj_details in details for pair, v in obj_details])
#         val_dict = dict(zip(all_vals, self.norm(all_vals)))
#         def swap_values(pair_ls, keep_top=2):
#             return sorted([(w, val_dict[v]) for w, v in pair_ls],
#                           key=lambda t: t[1],
#                           reverse=True)[:keep_top]
#         details.apply(swap_values)
#         details.name = "score details"
#         return scores, details


In [22]:
class PMI:
    def __init__(self, texts, **model_params):
        default_params = dict(ns=2, documents=texts, precompute_freqs=True)
        default_params.update(model_params)
        self.model = Ngram(**default_params)
        
    def pmi(self, pair_str):
        w1, w2 = pair_str.split()
        return (
                self.model.prob(pair_str, log=True) - 
                (self.model.prob(w1, log=True) + 
                 self.model.prob(w2, log=True))
               )

    def process_object(self, row):
        pairs, pmis = [], []
        for text in row:
            cur_pairs = list(self.model.iter_ngrams(text, padding=False))
            pairs.extend(cur_pairs)
            pmis.extend(map(self.pmi, cur_pairs))
            
        return sorted(zip(pairs, pmis), key=lambda t:t[1]), np.mean(pmis)
    
    
    # distribution at q=99.5 is very narrow
    # -> q=95 is broader and retains 90% of objects
    @staticmethod
    def percentile_norm(v, q=95):
        return (v - np.percentile(v, 100-q))/(np.percentile(v, q) - np.percentile(v, 100-q))

    # taking sqrt implies scores centered & symmetric around 0.5
    # (otherwise high skew towards 0)
    def norm(self, v, q=95, do_sqrt=False):
        normed = self.percentile_norm(v, q)
        return normed**0.5 if do_sqrt else normed
        
        
    
    def process_objects(self, rows):
        tuples = rows.progress_apply(
                    axis='columns', 
                    func=self.process_object
                )
        
        scores = tuples.apply(lambda t: t[1])
#         scores = self.norm(scores, do_sqrt=True)
        scores.name = "scores"
        
        
        details = tuples.apply(lambda t: t[0])
        all_vals = np.asarray([v for obj_details in details for pair, v in obj_details])
        val_dict = dict(zip(all_vals, self.norm(all_vals)))
        def swap_values(pair_ls, keep_top=2):
            return sorted([(w, val_dict[v]) for w, v in pair_ls],
                          key=lambda t: t[1],
                          reverse=True)[:keep_top]
        details.apply(swap_values)
        details.name = "score details"
        return scores, details


In [23]:
texts = list(df.Title.fillna("")) + list(df.Description.fillna(""))

c = PMI(texts=texts)

(1, 2)-grams: Padding documents...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293388/293388 [00:00<00:00, 1856939.41it/s]


(1, 2)-grams: Term Document Matrix constructed...
(1, 2)-grams: Term frequencies precomputed...
(1, 2)-grams: Init done


In [24]:
cur_data = df.fillna("")[["Title", "Description"]].iloc[:1000]
scores, details = c.process_objects(cur_data)

  0%|                                                                                                                                                                                                                | 0/1000 [00:00<?, ?it/s]/home/valentin/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/valentin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1609.41it/s]


In [37]:
np.percentile(scores, q=50)

nan

In [20]:
scores = pd.Series(np.random.random(50))

c.norm(scores)

0     0.151439
1     0.204967
2     0.889557
3     0.984853
4     0.645023
5     0.105363
6     0.360677
7     0.589574
8    -0.044660
9     0.062796
10    0.511570
11    0.923574
12    0.969598
13    0.829749
14    0.510157
15    1.114575
16    1.012393
17    1.060358
18   -0.068810
19    0.746265
20    0.416046
21    0.312242
22    0.028681
23    0.161858
24    0.272616
25    0.969801
26    0.164548
27    0.234259
28    0.028750
29   -0.023466
30    0.231591
31    0.777305
32    0.914978
33    0.309403
34    0.654025
35    0.902578
36    0.349992
37    0.662644
38    0.808083
39    0.783884
40    0.324111
41    0.905197
42    0.592580
43    0.571348
44    0.938963
45    0.037772
46    0.068983
47    0.963868
48    0.119221
49    0.717566
dtype: float64

---

In [7]:
scores = []
details = []
for i, r in tqdm(df.fillna("")[["Title", "Description"]].iterrows(), total=df.shape[0]):
    detail, score = c.process_object(r)
    scores.append(score)
    details.append(detail)

  0%|                                                                                                                                                                                                              | 0/146694 [00:00<?, ?it/s]/home/valentin/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/valentin/.local/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 12%|███████████████████████▌                                                                                                                                                                         | 17924/146694 [00:29<03:28, 616.46it/s]


KeyboardInterrupt: 

---
# normalising

In [ ]:
_=plt.hist(scores, bins=100)

In [ ]:
def percentile_norm(v, q=99.5):
    return (v - np.percentile(v, 100-q))/(np.percentile(v, q) - np.percentile(v, 100-q))

a = np.asarray(scores)
a = a[~np.isnan(a)]

plt.hist(percentile_norm(a, q=95)**0.5, bins=100)
plt.xlim((0, 1))

In [ ]:
normed = percentile_norm(a, q=95)

((normed <= 1) & (normed >= 0)).sum()/normed.shape[0]

## details

In [ ]:
all_vals = np.asarray([v for obj_details in details for pair, v in obj_details])

_=plt.hist(percentile_norm(all_vals, q=95), bins=100)
plt.xlim((0, 1))

In [ ]:
len(set(all_vals))/len(all_vals)

---
# other stuff

In [ ]:
w, u = "gevaarlijke handelsexpedities".split()
c.model.freq(f"{w} {u}"), c.model.freq(w), c.model.freq(u)